In [1]:
import numpy as np 
import pandas as pd 

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import nltk
from nltk.corpus import stopwords
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('spam.csv', encoding='latin1')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [5]:
df.tail()   

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN
5571,ham,Rofl. Its true to its name,NaN,NaN,NaN


In [6]:
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)

In [7]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [8]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [9]:
df.rename(columns={'v1': 'Classification', 'v2': 'SMS Text'}, inplace=True)
df.head()

,Classification,SMS Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
df['Classification'] = encoder.fit_transform(df['Classification'])


In [11]:
df.head()

,Classification,SMS Text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [12]:
df.duplicated().sum()

403

In [13]:
df = df.drop_duplicates(keep='first')

In [14]:
import nltk
df['num_char'] = df['SMS Text'].apply(len)

In [15]:
df.head()

,Classification,SMS Text,num_char
0,0,"Go until jurong point, crazy.. Available only ...",111
1,0,Ok lar... Joking wif u oni...,29
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,0,U dun say so early hor... U c already then say...,49
4,0,"Nah I don't think he goes to usf, he lives aro...",61


In [16]:
df.head()

,Classification,SMS Text,num_char
0,0,"Go until jurong point, crazy.. Available only ...",111
1,0,Ok lar... Joking wif u oni...,29
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,0,U dun say so early hor... U c already then say...,49
4,0,"Nah I don't think he goes to usf, he lives aro...",61


In [17]:
subset = df[df['Classification'] == 1]  
description = subset['Classification'].describe()  

print(description)

count    653.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: Classification, dtype: float64


In [18]:
subset2 = df[df['Classification'] == 0]  
description = subset['Classification'].describe()  

print(description)


count    653.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: Classification, dtype: float64


In [19]:
df.describe()

,Classification,num_char
count,5169.000000,5169.000000
mean,0.126330,78.977945
std,0.332253,58.236293
min,0.000000,2.000000
25%,0.000000,36.000000
50%,0.000000,60.000000
75%,0.000000,117.000000
max,1.000000,910.000000


In [20]:
mean_num_char = df.groupby('Classification')['num_char'].mean()

print(mean_num_char)

Classification
0     70.459256
1    137.891271
Name: num_char, dtype: float64


In [21]:
df.head()

,Classification,SMS Text,num_char
0,0,"Go until jurong point, crazy.. Available only ...",111
1,0,Ok lar... Joking wif u oni...,29
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,0,U dun say so early hor... U c already then say...,49
4,0,"Nah I don't think he goes to usf, he lives aro...",61


In [22]:
import seaborn as sns

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english')

spam_texts = df[df['Classification'] == 1]['SMS Text']
tfidf_matrix = tfidf_vectorizer.fit_transform(spam_texts)

feature_names = tfidf_vectorizer.get_feature_names_out()

mean_tfidf_scores = tfidf_matrix.mean(axis=0).A1

word_tfidf_df = pd.DataFrame({'word': feature_names, 'mean_tfidf_score': mean_tfidf_scores})

high_spam_words = word_tfidf_df.sort_values(by='mean_tfidf_score', ascending=False)

print(high_spam_words.head(50))

            word  mean_tfidf_score
1315        free          0.035586
1739      mobile          0.024252
2480         txt          0.023741
2523          ur          0.023206
2389        text          0.023152
1009       claim          0.022208
2300        stop          0.022183
2092       reply          0.021906
1998       prize          0.021738
2657         www          0.017787
1810         new          0.017562
2645         won          0.016809
961         cash          0.016809
2525      urgent          0.016528
1534        just          0.015950
1070     contact          0.015114
339         150p          0.015045
2182     service          0.014889
1821       nokia          0.014677
2496          uk          0.014328
2177        send          0.014077
2626         win          0.013131
1394  guaranteed          0.012646
1705     message          0.012461
2610        week          0.012347
1767         msg          0.012263
1917       phone          0.012235
1111    customer    

In [24]:
high_spam_words = word_tfidf_df.sort_values(by='mean_tfidf_score', ascending=False).head(20)

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X = df['SMS Text']
y = df['Classification']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9642166344294004

Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       889
           1       0.98      0.76      0.86       145

    accuracy                           0.96      1034
   macro avg       0.97      0.88      0.92      1034
weighted avg       0.96      0.96      0.96      1034


Confusion Matrix:
[[887   2]
 [ 35 110]]


In [26]:
example_message = ["Free entry prize in 2 a wkly comp to win FA Cup final tickets!"]

example_message_tfidf = tfidf_vectorizer.transform(example_message)

predicted_label = model.predict(example_message_tfidf)

predicted_class = 'ham' if predicted_label == 0 else 'spam'

print(f"The model predicts the example message as: {predicted_class}")

The model predicts the example message as: spam
